# CODE

In [3]:
!pip install keras-nightly

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models, optimizers, Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras import backend as K
import keras
from tensorflow.keras.models import load_model

In [ ]:
train_dir = "/kaggle/input/huong-thom/split_Huongthom/train"
valid_dir = "/kaggle/input/huong-thom/split_Huongthom/validation"
test_dir = "/kaggle/input/huong-thom/split_Huongthom/test"
# Compute mean and std
image_generator = ImageDataGenerator(rescale=1./255)  # Rescale pixel values to [0, 1] for computation
train_data = image_generator.flow_from_directory(train_dir, target_size=(75, 224), batch_size=32, class_mode=None, shuffle=False)

x = np.concatenate([train_data[i] for i in range(len(train_data))], axis=0)
mean = np.mean(x, axis=(0, 1, 2))
std = np.std(x, axis=(0, 1, 2))

print("mean:", mean)
print("std:", std)

# Create data generators with the calculated mean and std for normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=15,
    featurewise_center=True,
    featurewise_std_normalization=True
)
train_datagen.mean = mean  # Set the mean for normalization
train_datagen.std = std    # Set the std for normalization

valid_datagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True
)

valid_datagen.mean = mean  
valid_datagen.std = std   

test_datagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True
)

test_datagen.mean = mean  
test_datagen.std = std  

train_loader = train_datagen.flow_from_directory(train_dir, target_size=(75, 224), batch_size=32, class_mode='binary', shuffle=True)
valid_loader = valid_datagen.flow_from_directory(valid_dir, target_size=(75, 224), batch_size=32, class_mode='binary', shuffle=False)
test_loader = test_datagen.flow_from_directory(test_dir, target_size=(75, 224), batch_size=32, class_mode='binary', shuffle=False)

Found 2488 images belonging to 2 classes.
mean: [0.40138417 0.40138417 0.30693865]
std: [0.22504932 0.1927605  0.1994153 ]
Found 2488 images belonging to 2 classes.
Found 830 images belonging to 2 classes.
Found 832 images belonging to 2 classes.


In [ ]:
def resnet_block(inputs, filters, kernel_size=3, stride=1, activation="relu", block_name="block"):
    # Xavier initializers
    initializer = tf.keras.initializers.GlorotUniform()

    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding="same", 
                      kernel_initializer=initializer, name=f"{block_name}_conv1")(inputs)
    x = layers.BatchNormalization(name=f"{block_name}_bn1")(x)
    x = layers.Activation(activation, name=f"{block_name}_act1")(x)

    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=1, padding="same", 
                      kernel_initializer=initializer, name=f"{block_name}_conv2")(x)
    x = layers.BatchNormalization(name=f"{block_name}_bn2")(x)

    shortcut = inputs

    if stride != 1 or inputs.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=1, strides=stride, padding="same", 
                                  kernel_initializer=initializer, name=f"{block_name}_shortcut_conv")(inputs)
        shortcut = layers.BatchNormalization(name=f"{block_name}_shortcut_bn")(shortcut)

    x = layers.add([x, shortcut], name=f"{block_name}_add")
    x = layers.Activation(activation, name=f"{block_name}_act2")(x)
    
    return x

def build_resnet18(input_shape=(75, 224, 3), num_classes=1):
    inputs = layers.Input(shape=input_shape, name="input_layer")

    # Initial Conv layer with Xavier initialization
    initializer = tf.keras.initializers.GlorotUniform()
    x = layers.Conv2D(64, kernel_size=7, strides=2, padding="same", 
                      kernel_initializer=initializer, name="conv_initial")(inputs)
    x = layers.BatchNormalization(name="bn_initial")(x)
    x = layers.Activation("relu", name="act_initial")(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding="same", name="maxpool_initial")(x)

    # Residual blocks
    x = resnet_block(x, 64, stride=1, block_name="block1_1")
    x = resnet_block(x, 64, stride=1, block_name="block1_2")
    x = resnet_block(x, 128, stride=2, block_name="block2_1")
    x = resnet_block(x, 128, stride=1, block_name="block2_2")

    x = resnet_block(x, 256, stride=2, block_name="block3_1")
    x = resnet_block(x, 256, stride=1, block_name="block3_2")

    x = resnet_block(x, 512, stride=2, block_name="block4_1")
    x = resnet_block(x, 512, stride=1, block_name="block4_2")

    # Global Average Pooling and Output Layer
    x = layers.GlobalAveragePooling2D(name="global_avg_pool")(x)
    outputs = layers.Dense(num_classes, activation="sigmoid", name="output_layer")(x)  
    model = models.Model(inputs, outputs, name="ResNet18")

    return model

resnet18 = build_resnet18(input_shape=(75, 224, 3), num_classes=1) 
resnet18.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
resnet18.summary()

Model: "ResNet18"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 75, 224,   │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_initial        │ (None, 38, 112,   │      9,472 │ input_layer[0][0] │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_initial          │ (None, 38, 112,   │        256 │ conv_initial[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ act_initial         │ (None, 38, 112,   │          0 │ bn_initial[0][0]  │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ maxpool_initial     │ (None, 19, 56,    │          0 │ act_initial[0][0] │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_conv1      │ (None, 19, 56,    │     36,928 │ maxpool_initial[… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_bn1        │ (None, 19, 56,    │        256 │ block1_1_conv1[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_act1       │ (None, 19, 56,    │          0 │ block1_1_bn1[0][… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_conv2      │ (None, 19, 56,    │     36,928 │ block1_1_act1[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_bn2        │ (None, 19, 56,    │        256 │ block1_1_conv2[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_add (Add)  │ (None, 19, 56,    │          0 │ block1_1_bn2[0][… │
│                     │ 64)               │            │ maxpool_initial[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_act2       │ (None, 19, 56,    │          0 │ block1_1_add[0][… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_conv1      │ (None, 19, 56,    │     36,928 │ block1_1_act2[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_bn1        │ (None, 19, 56,    │        256 │ block1_2_conv1[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_act1       │ (None, 19, 56,    │          0 │ block1_2_bn1[0][… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_conv2      │ (None, 19, 56,    │     36,928 │ block1_2_act1[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_bn2        │ (None, 19, 56,    │        256 │ block1_2_conv2[0

 Total params: 11,191,425 (42.69 MB)

 Trainable params: 11,181,825 (42.66 MB)

 Non-trainable params: 9,600 (37.50 KB)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
history = resnet18.fit(
    train_loader,
    validation_data=valid_loader,
    epochs=100,
    callbacks=[early_stopping]
)

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100


I0000 00:00:1730224930.274501     116 service.cc:145] XLA service 0x7f9760003470 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730224930.274567     116 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1730224930.274573     116 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 2/78 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - accuracy: 0.4922 - loss: 0.7571  

I0000 00:00:1730224941.578984     116 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step - accuracy: 0.7764 - loss: 0.4551

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 404ms/step - accuracy: 0.7773 - loss: 0.4536 - val_accuracy: 0.5048 - val_loss: 0.8485
Epoch 2/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 204ms/step - accuracy: 0.9224 - loss: 0.2082 - val_accuracy: 0.5048 - val_loss: 0.9273
Epoch 3/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 201ms/step - accuracy: 0.9460 - loss: 0.1382 - val_accuracy: 0.5048 - val_loss: 1.5937
Epoch 4/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 17s 211ms/step - accuracy: 0.9556 - loss: 0.1101 - val_accuracy: 0.8590 - val_loss: 0.3329
Epoch 5/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 204ms/step - accuracy: 0.9669 - loss: 0.0931 - val_accuracy: 0.5193 - val_loss: 1.8291
Epoch 6/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 205ms/step - accuracy: 0.9733 - loss: 0.0649 - val_accuracy: 0.9120 - val_loss: 0.2301
Epoch 7/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 208ms/step - accuracy: 0.9615 - loss: 0.0969 - val_accuracy: 0.9614 - val_loss: 0.1434
Epoch 8/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 209ms/step - accuracy: 0.9682 - loss: 0.0820 - val_accuracy

In [8]:
resnet18.save('./teacher_model.h5')

In [ ]:
loaded_teacher_model = load_model('teacher_model.h5')
loaded_teacher_model.evaluate(test_loader)

26/26 ━━━━━━━━━━━━━━━━━━━━ 7s 194ms/step - accuracy: 0.9794 - loss: 0.0538


[0.05163998901844025, 0.9807692170143127]

In [ ]:
class CBAM(layers.Layer):
    def __init__(self, ratio=16,**kwargs):
        super(CBAM, self).__init__(**kwargs)
        self.ratio = ratio

    def build(self, input_shape):
        self.channels = input_shape[-1]
        self.shared_dense_one = layers.Dense(self.channels // self.ratio, activation='relu', use_bias=False)
        self.shared_dense_two = layers.Dense(self.channels, use_bias=False)
        self.spatial_conv = layers.Conv2D(1, kernel_size=7, padding="same", activation="sigmoid")

    def channel_attention(self, inputs):
        avg_pool = layers.GlobalAveragePooling2D()(inputs)
        avg_fc = self.shared_dense_two(self.shared_dense_one(avg_pool))
        max_pool = layers.GlobalMaxPooling2D()(inputs)
        max_fc = self.shared_dense_two(self.shared_dense_one(max_pool))
        channel_attention = layers.Activation("sigmoid")(avg_fc + max_fc)
        refined = layers.Multiply()([inputs, channel_attention])
        return refined

    def spatial_attention(self, inputs):
        avg_pool = tf.reduce_mean(inputs, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(inputs, axis=-1, keepdims=True)
        concat = layers.Concatenate()([avg_pool, max_pool])
        spatial_attention = self.spatial_conv(concat)
        refined = layers.Multiply()([inputs, spatial_attention])
        return refined

    def call(self, inputs):
        x = self.channel_attention(inputs)
        x = self.spatial_attention(x)
        return x

class DistillationBlock(layers.Layer):
    def __init__(self, filters=64, **kwargs):
        super(DistillationBlock, self).__init__(**kwargs)
        self.filters = filters
        self.depthwise_conv1 = layers.SeparableConv2D(
            filters=filters, kernel_size=3, padding='same', strides=2, depth_multiplier=1
        )
        self.depthwise_conv2 = layers.SeparableConv2D(
            filters=filters, kernel_size=3, padding='same', strides=2, depth_multiplier=2
        )
        self.bn1 = layers.BatchNormalization()
        self.bn2 = layers.BatchNormalization()
        self.relu = layers.ReLU()
        
        self.conv = layers.Conv2D(
            filters=filters, kernel_size=3, padding='same', strides=2
        ) 
        self.bn_conv = layers.BatchNormalization()
        self.relu_conv = layers.ReLU()
        
        self.cbam = CBAM()

    def call(self, inputs):
        # First branch
        x1 = self.depthwise_conv1(inputs)
        x1 = self.bn1(x1)
        x1 = self.relu(x1)
        x1 = self.depthwise_conv2(x1)
        x1 = self.bn2(x1)
        x1 = self.relu(x1)
        x1 = self.cbam(x1)

        # Second branch
        x2 = self.conv(inputs)
        x2 = self.bn_conv(x2)
        x2 = self.relu_conv(x2)

        # Resize x1 to match x2's spatial dimensions
        x1 = tf.image.resize(x1, tf.shape(x2)[1:3])

        # Concatenate both branches
        output = layers.Concatenate()([x1, x2])
        
        return output

    def get_config(self):
        config = super(DistillationBlock, self).get_config()
        config.update({"filters": self.filters})
        return config

In [ ]:
teacher = load_model('teacher_model.h5')

In [ ]:
student_base_model_c1 = load_model('teacher_model.h5')
cbam = CBAM()
layer_name = 'block1_2_act2'

base_output = student_base_model_c1.get_layer(layer_name).output
cbam_output = cbam(base_output)
distillation_output1 = DistillationBlock(filters=64)(cbam_output)
distillation_output2 = DistillationBlock(filters=128)(distillation_output1)
distillation_output3 = DistillationBlock(filters=256)(distillation_output2)
X = layers.Flatten()(distillation_output3)  # Use distillation_output3 instead of distillation_output2 for more depth
X = layers.Dense(1, activation="sigmoid", name="output_layer")(X)

student_model_c1 = Model(inputs=student_base_model_c1.input, outputs=X)  # Use the base model's input

for layer in student_base_model_c1.layers:
    if layer.name not in [layer_name]:
        layer.trainable = False

teacher_features = Model(inputs=teacher.input, outputs=teacher.get_layer(layer_name).output)
student_features = Model(inputs=student_base_model_c1.input, outputs=student_base_model_c1.get_layer(layer_name).output)


student_model_c1.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
                         loss='binary_crossentropy',
                         metrics=['accuracy'])

student_model_c1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 75, 224,   │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_initial        │ (None, 38, 112,   │      9,472 │ input_layer[0][0] │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_initial          │ (None, 38, 112,   │        256 │ conv_initial[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ act_initial         │ (None, 38, 112,   │          0 │ bn_initial[0][0]  │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ maxpool_initial     │ (None, 19, 56,    │          0 │ act_initial[0][0] │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_conv1      │ (None, 19, 56,    │     36,928 │ maxpool_initial[… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_bn1        │ (None, 19, 56,    │        256 │ block1_1_conv1[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_act1       │ (None, 19, 56,    │          0 │ block1_1_bn1[0][… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_conv2      │ (None, 19, 56,    │     36,928 │ block1_1_act1[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_bn2        │ (None, 19, 56,    │        256 │ block1_1_conv2[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_add (Add)  │ (None, 19, 56,    │          0 │ block1_1_bn2[0][… │
│                     │ 64)               │            │ maxpool_initial[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_act2       │ (None, 19, 56,    │          0 │ block1_1_add[0][… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_conv1      │ (None, 19, 56,    │     36,928 │ block1_1_act2[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_bn1        │ (None, 19, 56,    │        256 │ block1_2_conv1[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_act1       │ (None, 19, 56,    │          0 │ block1_2_bn1[0][… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_conv2      │ (None, 19, 56,    │     36,928 │ block1_2_act1[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_bn2        │ (None, 19, 56,    │        256 │ block1_2_conv2[0

 Total params: 1,231,885 (4.70 MB)

 Trainable params: 1,070,733 (4.08 MB)

 Non-trainable params: 161,152 (629.50 KB)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
student_history_without_loss = student_model_c1.fit(
    train_loader,
    validation_data=valid_loader,
    epochs=100,
    callbacks=[early_stopping]
)

Epoch 1/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 51s 392ms/step - accuracy: 0.8774 - loss: 0.2644 - val_accuracy: 0.9181 - val_loss: 0.3735
Epoch 2/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 17s 213ms/step - accuracy: 0.9664 - loss: 0.0911 - val_accuracy: 0.9675 - val_loss: 0.1821
Epoch 3/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 17s 216ms/step - accuracy: 0.9800 - loss: 0.0663 - val_accuracy: 0.9747 - val_loss: 0.1078
Epoch 4/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 17s 215ms/step - accuracy: 0.9845 - loss: 0.0455 - val_accuracy: 0.9614 - val_loss: 0.1156
Epoch 5/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 201ms/step - accuracy: 0.9751 - loss: 0.0544 - val_accuracy: 0.9759 - val_loss: 0.0673
Epoch 6/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 209ms/step - accuracy: 0.9788 - loss: 0.0503 - val_accuracy: 0.9783 - val_loss: 0.0800
Epoch 7/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 210ms/step - accuracy: 0.9869 - loss: 0.0381 - val_accuracy: 0.9807 - val_loss: 0.0590
Epoch 8/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 206ms/step - accuracy: 0.9860 - loss: 0.0333 - 

In [14]:
student_model_c1.evaluate(test_loader)

26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - accuracy: 0.9940 - loss: 0.0173


[0.021492507308721542, 0.9915865659713745]

In [15]:
student_model_c1.save('./student_c1_model.h5')

In [ ]:
student_base_model_c2 = load_model('teacher_model.h5')
cbam = CBAM()
layer_name = 'block2_2_act2'
base_output = student_base_model_c2.get_layer(layer_name).output
cbam_output = cbam(base_output)
distillation_output1 = DistillationBlock(filters=128)(cbam_output)
distillation_output2 = DistillationBlock(filters=256)(distillation_output1)

X = layers.Flatten()(distillation_output2)  # Use distillation_output3 instead of distillation_output2 for more depth
X = layers.Dense(1, activation="sigmoid", name="output_layer")(X)

student_model_c2 = Model(inputs=student_base_model_c2.input, outputs=X)  # Use the base model's input
for layer in student_base_model_c2.layers:
    if layer.name != layer_name:
        layer.trainable = False

student_model_c2.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
                            loss='binary_crossentropy',
                            metrics=['accuracy'])

student_model_c2.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 75, 224,   │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_initial        │ (None, 38, 112,   │      9,472 │ input_layer[0][0] │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_initial          │ (None, 38, 112,   │        256 │ conv_initial[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ act_initial         │ (None, 38, 112,   │          0 │ bn_initial[0][0]  │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ maxpool_initial     │ (None, 19, 56,    │          0 │ act_initial[0][0] │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_conv1      │ (None, 19, 56,    │     36,928 │ maxpool_initial[… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_bn1        │ (None, 19, 56,    │        256 │ block1_1_conv1[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_act1       │ (None, 19, 56,    │          0 │ block1_1_bn1[0][… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_conv2      │ (None, 19, 56,    │     36,928 │ block1_1_act1[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_bn2        │ (None, 19, 56,    │        256 │ block1_1_conv2[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_add (Add)  │ (None, 19, 56,    │          0 │ block1_1_bn2[0][… │
│                     │ 64)               │            │ maxpool_initial[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_act2       │ (None, 19, 56,    │          0 │ block1_1_add[0][… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_conv1      │ (None, 19, 56,    │     36,928 │ block1_1_act2[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_bn1        │ (None, 19, 56,    │        256 │ block1_2_conv1[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_act1       │ (None, 19, 56,    │          0 │ block1_2_bn1[0][… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_conv2      │ (None, 19, 56,    │     36,928 │ block1_2_act1[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_bn2        │ (None, 19, 56,    │        256 │ block1_2_conv2[0

 Total params: 1,708,458 (6.52 MB)

 Trainable params: 1,020,202 (3.89 MB)

 Non-trainable params: 688,256 (2.63 MB)

In [ ]:
student_c2 = student_model_c2.fit(
    train_loader,
    validation_data=valid_loader,
    epochs=100,
    callbacks=[early_stopping]
)

Epoch 1/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 36s 297ms/step - accuracy: 0.8934 - loss: 0.2430 - val_accuracy: 0.9747 - val_loss: 0.5624
Epoch 2/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 187ms/step - accuracy: 0.9814 - loss: 0.0493 - val_accuracy: 0.9807 - val_loss: 0.4070
Epoch 3/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 191ms/step - accuracy: 0.9889 - loss: 0.0317 - val_accuracy: 0.9819 - val_loss: 0.2275
Epoch 4/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 192ms/step - accuracy: 0.9908 - loss: 0.0321 - val_accuracy: 0.9843 - val_loss: 0.1184
Epoch 5/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 196ms/step - accuracy: 0.9934 - loss: 0.0238 - val_accuracy: 0.9843 - val_loss: 0.0691
Epoch 6/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 188ms/step - accuracy: 0.9954 - loss: 0.0197 - val_accuracy: 0.9867 - val_loss: 0.0517
Epoch 7/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 192ms/step - accuracy: 0.9952 - loss: 0.0171 - val_accuracy: 0.9867 - val_loss: 0.0482
Epoch 8/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 192ms/step - accuracy: 0.9896 - loss: 0.0266 - 

In [18]:
student_model_c2.evaluate(test_loader)

26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.9923 - loss: 0.0265


[0.021270904690027237, 0.9939903616905212]

In [19]:
student_model_c2.save('./student_c2_model.h5')

In [ ]:
student_base_model_c3 = load_model('teacher_model.h5')
cbam = CBAM()
layer_name = 'block3_2_act2'
base_output = student_base_model_c3.get_layer(layer_name).output
cbam_output = cbam(base_output)
distillation_output = DistillationBlock(filters=256)(cbam_output)

X = layers.Flatten()(distillation_output)
X = layers.Dense(1, activation="sigmoid", name="output_layer")(X)
student_model_c3 = Model(inputs=student_base_model_c3.input, outputs=X)

for layer in student_base_model_c3.layers:
    if layer.name not in [layer_name]:
        layer.trainable = False

student_model_c3.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
                            loss='binary_crossentropy',
                            metrics=['accuracy'])

student_model_c3.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 75, 224,   │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_initial        │ (None, 38, 112,   │      9,472 │ input_layer[0][0] │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_initial          │ (None, 38, 112,   │        256 │ conv_initial[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ act_initial         │ (None, 38, 112,   │          0 │ bn_initial[0][0]  │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ maxpool_initial     │ (None, 19, 56,    │          0 │ act_initial[0][0] │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_conv1      │ (None, 19, 56,    │     36,928 │ maxpool_initial[… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_bn1        │ (None, 19, 56,    │        256 │ block1_1_conv1[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_act1       │ (None, 19, 56,    │          0 │ block1_1_bn1[0][… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_conv2      │ (None, 19, 56,    │     36,928 │ block1_1_act1[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_bn2        │ (None, 19, 56,    │        256 │ block1_1_conv2[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_add (Add)  │ (None, 19, 56,    │          0 │ block1_1_bn2[0][… │
│                     │ 64)               │            │ maxpool_initial[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_1_act2       │ (None, 19, 56,    │          0 │ block1_1_add[0][… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_conv1      │ (None, 19, 56,    │     36,928 │ block1_1_act2[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_bn1        │ (None, 19, 56,    │        256 │ block1_2_conv1[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_act1       │ (None, 19, 56,    │          0 │ block1_2_bn1[0][… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_conv2      │ (None, 19, 56,    │     36,928 │ block1_2_act1[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_2_bn2        │ (None, 19, 56,    │        256 │ block1_2_conv2[0

 Total params: 3,614,023 (13.79 MB)

 Trainable params: 822,983 (3.14 MB)

 Non-trainable params: 2,791,040 (10.65 MB)

In [ ]:
student_c3 = student_model_c3.fit(
    train_loader,
    validation_data=valid_loader,
    epochs=100,
    callbacks=[early_stopping]
)

Epoch 1/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 30s 270ms/step - accuracy: 0.9363 - loss: 0.1761 - val_accuracy: 0.9819 - val_loss: 0.0700
Epoch 2/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 187ms/step - accuracy: 0.9943 - loss: 0.0220 - val_accuracy: 0.9843 - val_loss: 0.0457
Epoch 3/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 189ms/step - accuracy: 0.9950 - loss: 0.0165 - val_accuracy: 0.9819 - val_loss: 0.0451
Epoch 4/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 188ms/step - accuracy: 0.9971 - loss: 0.0113 - val_accuracy: 0.9843 - val_loss: 0.0420
Epoch 5/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 194ms/step - accuracy: 0.9965 - loss: 0.0097 - val_accuracy: 0.9843 - val_loss: 0.0437
Epoch 6/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 14s 185ms/step - accuracy: 0.9983 - loss: 0.0090 - val_accuracy: 0.9843 - val_loss: 0.0476
Epoch 7/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 188ms/step - accuracy: 0.9961 - loss: 0.0108 - val_accuracy: 0.9855 - val_loss: 0.0498
Epoch 8/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 15s 190ms/step - accuracy: 0.9974 - loss: 0.0150 - 

In [22]:
student_model_c3.evaluate(test_loader)

26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.9926 - loss: 0.0253


[0.02616506814956665, 0.9915865659713745]

In [23]:
student_model_c3.save('./student_c3_model.h5')

In [24]:
class MultiStudentDistiller(keras.Model):
    def __init__(self, students, teacher):
        """
        teacher: The teacher model
        students: A list of student models
        """
        super(MultiStudentDistiller, self).__init__()
        self.teacher = teacher
        self.students = students

    def compile(self, optimizers, metrics, student_loss_fn, distillation_loss_fn, alpha=0.1, temperature=3):
        # Store individual optimizers and metrics for each student
        self.optimizers = optimizers
        self.student_metrics = metrics  # Renamed from 'self.metrics' to avoid conflict
        self.val_metrics = [m.__class__() for m in metrics]

        # Set losses and distillation parameters
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

        super(MultiStudentDistiller, self).compile()

    def train_step(self, data):
        x, y = data
        student_losses, distillation_losses = [], []

        # Calculate losses for each student
        for i, student in enumerate(self.students):
            with tf.GradientTape() as tape:
                student_preds = student(x, training=True)
                teacher_preds = self.teacher(x, training=False)

                # Compute student and distillation losses
                student_loss = self.student_loss_fn(y, student_preds)
                distillation_loss = self.distillation_loss_fn(
                    tf.nn.softmax(teacher_preds / self.temperature, axis=1),
                    tf.nn.softmax(student_preds / self.temperature, axis=1)
                )
                loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

            # Update each student model
            grads = tape.gradient(loss, student.trainable_variables)
            self.optimizers[i].apply_gradients(zip(grads, student.trainable_variables))

            # Record losses
            student_losses.append(student_loss)
            distillation_losses.append(distillation_loss)

        # Ensure all lists have correct lengths
        assert len(student_losses) == len(self.students), "Mismatch in student_losses length"
        assert len(distillation_losses) == len(self.students), "Mismatch in distillation_losses length"

        # Update metrics and create results dictionary
        results = {f"student_{i+1}_accuracy": metric(y, student(x, training=False)) 
                   for i, metric in enumerate(self.student_metrics)}
        for i in range(len(self.students)):
            results[f"student_{i+1}_student_loss"] = student_losses[i]
            results[f"student_{i+1}_distillation_loss"] = distillation_losses[i]

        return results

    def test_step(self, data):
        x, y = data
        student_val_losses = []
        results = {}

        for i, student in enumerate(self.students):
            y_prediction = student(x, training=False)
            student_loss = self.student_loss_fn(y, y_prediction)

            # Update validation metric and loss
            self.val_metrics[i].update_state(y, y_prediction)
            student_val_losses.append(student_loss)
            results[f"student_{i+1}_val_accuracy"] = self.val_metrics[i].result()
            results[f"student_{i+1}_val_loss"] = student_loss

        return results

    def predict_step(self, data):
        x, _ = data
        predictions = {}
        for i, student in enumerate(self.students):
            predictions[f"student_{i+1}_predictions"] = student(x, training=False)
        return predictions

In [25]:
# Custom objects dictionary
custom_objects = {"DistillationBlock": DistillationBlock, "CBAM": CBAM}

# Load the models with custom_objects
teacher_model = load_model('teacher_model.h5', custom_objects=custom_objects)
student_model_1 = load_model('student_c1_model.h5', custom_objects=custom_objects)
student_model_2 = load_model('student_c2_model.h5', custom_objects=custom_objects)
student_model_3 = load_model('student_c3_model.h5', custom_objects=custom_objects)

# Initialize the distiller with three students
distiller = MultiStudentDistiller(students=[student_model_1, student_model_2, student_model_3], teacher=teacher_model)

# Compile the distiller with separate optimizers and metrics for each student
distiller.compile(
    optimizers=[tf.keras.optimizers.Adam(), tf.keras.optimizers.Adam(), tf.keras.optimizers.Adam()],
    metrics=[
        tf.keras.metrics.BinaryAccuracy(), 
        tf.keras.metrics.BinaryAccuracy(), 
        tf.keras.metrics.BinaryAccuracy()
    ],
    student_loss_fn=tf.keras.losses.BinaryCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
distiller.summary()

Model: "multi_student_distiller"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ResNet18 (Functional)           │ (None, 1)              │    11,191,425 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional (Functional)         │ (None, 1)              │     1,231,885 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_3 (Functional)       │ (None, 1)              │     1,708,458 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_4 (Functional)       │ (None, 1)              │     3,614,023 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,745,791 (67.69 MB)

 Trainable params: 14,095,743 (53.77 MB)

 Non-trainable params: 3,650,048 (13.92 MB)

In [ ]:
history = distiller.fit(
    train_loader,
    validation_data=valid_loader,
    epochs=30
)

Epoch 1/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 92s 542ms/step - student_1_accuracy: 0.9972 - student_1_distillation_loss: 0.0000e+00 - student_1_student_loss: 0.1012 - student_2_accuracy: 0.9972 - student_2_distillation_loss: 0.0000e+00 - student_2_student_loss: 0.0301 - student_3_accuracy: 0.9972 - student_3_distillation_loss: 0.0000e+00 - student_3_student_loss: 0.0339 - val_student_1_val_accuracy: 0.9337 - val_student_1_val_loss: 1.7017e-05 - val_student_2_val_accuracy: 0.9880 - val_student_2_val_loss: 4.9022e-07 - val_student_3_val_accuracy: 0.9783 - val_student_3_val_loss: 9.0500e-07
Epoch 2/30
78/78 ━━━━━━━━━━━━━━━━━━━━ 16s 208ms/step - student_1_accuracy: 0.9938 - student_1_distillation_loss: 0.0000e+00 - student_1_student_loss: 0.0525 - student_2_accuracy: 0.9938 - student_2_distillation_loss: 0.0000e+00 - student_2_student_loss: 0.0263 - student_3_accuracy: 0.9938 - student_3_distillation_loss: 0.0000e+00 - student_3_student_loss: 0.0181 - val_student_1_val_accuracy: 0.9145 - val_stude

In [27]:
distiller.evaluate(test_loader)

26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - student_1_val_accuracy: 0.9724 - student_1_val_loss: 0.0524 - student_2_val_accuracy: 0.9888 - student_2_val_loss: 0.0295 - student_3_val_accuracy: 0.9968 - student_3_val_loss: 0.0265


[<tf.Tensor: shape=(), dtype=float32, numpy=0.98197114>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.4095116e-08>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.99158657>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00094120385>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.9963942>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.009242624>]